In [11]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
X = np.array([[-1, -2], [-1, -4]])
clustering = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='average', distance_threshold=-0.1).fit(X)

In [1]:
import numpy as np
import json
import codecs
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from model import my_aggclustering

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


IndentationError: unexpected indent (model.py, line 79)

In [2]:
msdat_dir = '/home/hpms/Microsoft.IoT-Dump1.json'
with codecs.open(msdat_dir, 'r', 'utf-8-sig') as f:
    msdat = json.load(f)
msdat = pd.DataFrame(msdat)
msdat['Commands'] = [tuple(session) for session in msdat['Commands']]
msdat = msdat.drop_duplicates(subset='Commands').reset_index(drop=True)  # drop duplicates
msdat['Commands'] = [list(session) for session in msdat['Commands']]
msdat['Commands Length'] = [len(session) for session in msdat['Commands']]
msdat = msdat.sort_values(by='FirstSeen').reset_index(drop=True)

In [7]:
def find_score(score_list, depth):
    return np.mean([score[-1] if len(score)<depth else score[depth-1] for score in score_list])

### Baseline

$\alpha_{cluster} = \alpha_{label} = 0.1$,
No clustering.
12 length of sequence

In [3]:
n_session = len(msdat)
my_agg = my_aggclustering(msdat[:int(n_session*0.9)], 0.1, -1, 0.1, 1)
my_agg.fit()
with open('data/aggclustering2_9010_baseline.pkl', 'wb') as f:
    pickle.dump(my_agg, f)

Depth 0 started.
Depth 0 finished. Time spent: 186.47s.


In [4]:
with open('data/aggclustering2_9010_baseline.pkl', 'rb') as f:
    my_agg = pickle.load(f)
clustering = my_agg.memory['clustering']
init_commands_lists = my_agg.memory['init_commands_lists']
posterior_dicts = my_agg.memory['posterior_dicts']
expected_predictive_prob = my_agg.memory['expected_predictive_prob']
A_list = my_agg.memory['A_list']
E_list = my_agg.memory['E_list']
seq_rank_list = my_agg.memory['seq_rank_list']
cluster_num = my_agg.memory['cluster_num']

In [5]:
score_list = my_agg.predict(msdat[int(n_session*0.9):])

In [11]:
print('Average score for baseline is {:.5f}.'.format(find_score(score_list, 1)))

Average score for baseline is 0.28793.


### First

$\alpha_{cluster} = \alpha_{label} = 0.1$

$Decision Threshold = 0.5$

$Maximum Depth = 4$

In [4]:
my_agg = my_aggclustering(msdat[:int(n_session*0.9)], 0.1, 0.5, 0.1, 4)
my_agg.fit()
with open('data/aggclustering2_9010.pkl', 'wb') as f:
    pickle.dump(my_agg, f)

Depth 0 started.
Depth 0 finished. Time spent: 74.99s.
Depth 1 started.
Depth 1 finished. Time spent: 681.88s.
Depth 2 started.
Depth 2 finished. Time spent: 1120.08s.
Depth 3 started.
Depth 3 finished. Time spent: 1392.97s.


In [31]:
with open('data/aggclustering2_9010.pkl', 'rb') as f:
    my_agg = pickle.load(f)
clustering = my_agg.memory['clustering']
init_commands_lists = my_agg.memory['init_commands_lists']
posterior_dicts = my_agg.memory['posterior_dicts']
expected_predictive_prob = my_agg.memory['expected_predictive_prob']
A_list = my_agg.memory['A_list']
E_list = my_agg.memory['E_list']
seq_rank_list = my_agg.memory['seq_rank_list']
cluster_num = my_agg.memory['cluster_num']

In [13]:
score_list = my_agg.predict(msdat[int(n_session*0.9):])

In [17]:
print('Average score for ', end='')
for depth in range(1,5):
    print('depth {}: {:.5f},'.format(depth-1, find_score(score_list, depth)), end=' ')

Average score for depth 0: 0.28989, depth 1: 0.44354, depth 2: 0.46997, depth 3: 0.49107, 

### Second

$\alpha_{cluster} = \alpha_{label} = 0.01$

$Decision Threshold = 0.5$

$Maximum Depth = 4$

In [11]:
my_agg = my_aggclustering(msdat[:int(n_session*0.9)], 0.01, 0.5, 0.01, 4)
my_agg.fit()
with open('data/aggclustering2_9010_0.01.pkl', 'wb') as f:
    pickle.dump(my_agg, f)

Depth 0 started.
Depth 0 finished. Time spent: 67.86s.
Depth 1 started.
Depth 1 finished. Time spent: 558.24s.
Depth 2 started.
Depth 2 finished. Time spent: 891.77s.
Depth 3 started.
Depth 3 finished. Time spent: 1109.73s.


In [18]:
with open('data/aggclustering2_9010_0.01.pkl', 'rb') as f:
    my_agg = pickle.load(f)
clustering = my_agg.memory['clustering']
init_commands_lists = my_agg.memory['init_commands_lists']
posterior_dicts = my_agg.memory['posterior_dicts']
expected_predictive_prob = my_agg.memory['expected_predictive_prob']
A_list = my_agg.memory['A_list']
E_list = my_agg.memory['E_list']
seq_rank_list = my_agg.memory['seq_rank_list']
cluster_num = my_agg.memory['cluster_num']

In [19]:
score_list = my_agg.predict(msdat[int(n_session*0.9):])

In [20]:
print('Average score for ', end='')
for depth in range(1,5):
    print('depth {}: {:.5f},'.format(depth-1, find_score(score_list, depth)), end=' ')

Average score for depth 0: 0.29089, depth 1: 0.44791, depth 2: 0.47500, depth 3: 0.49762, 

### Third

$\alpha_{cluster} = \alpha_{label} = 1$

$Decision Threshold = 0.5$

$Maximum Depth = 4$

In [22]:
my_agg = my_aggclustering(msdat[:int(n_session*0.9)], 1, 0.5, 1, 4)
my_agg.fit()
with open('data/aggclustering2_9010_1.pkl', 'wb') as f:
    pickle.dump(my_agg, f)

Depth 0 started.
Depth 0 finished. Time spent: 62.08s.
Depth 1 started.
Depth 1 finished, no further depth required. Time spent: 518.35s.


In [23]:
with open('data/aggclustering2_9010_1.pkl', 'rb') as f:
    my_agg = pickle.load(f)
clustering = my_agg.memory['clustering']
init_commands_lists = my_agg.memory['init_commands_lists']
posterior_dicts = my_agg.memory['posterior_dicts']
expected_predictive_prob = my_agg.memory['expected_predictive_prob']
A_list = my_agg.memory['A_list']
E_list = my_agg.memory['E_list']
seq_rank_list = my_agg.memory['seq_rank_list']
cluster_num = my_agg.memory['cluster_num']

In [24]:
score_list = my_agg.predict(msdat[int(n_session*0.9):])

In [25]:
print('Average score for ', end='')
for depth in range(1,5):
    print('depth {}: {:.5f},'.format(depth-1, find_score(score_list, depth)), end=' ')

Average score for depth 0: 0.28102, depth 1: 0.32444, depth 2: 0.32444, depth 3: 0.32444, 

### Fourth

$\alpha_{cluster} = \alpha_{label} = 0.1$

$Decision Threshold = 0.3$

$Maximum Depth = 4$

In [26]:
my_agg = my_aggclustering(msdat[:int(n_session*0.9)], 0.1, 0.3, 0.1, 4)
my_agg.fit()
with open('data/aggclustering2_9010_0.3dt.pkl', 'wb') as f:
    pickle.dump(my_agg, f)

Depth 0 started.
Depth 0 finished. Time spent: 68.44s.
Depth 1 started.
Depth 1 finished. Time spent: 553.48s.
Depth 2 started.
Depth 2 finished. Time spent: 882.74s.
Depth 3 started.
Depth 3 finished. Time spent: 1096.25s.


In [27]:
with open('data/aggclustering2_9010_0.3dt.pkl', 'rb') as f:
    my_agg = pickle.load(f)
clustering = my_agg.memory['clustering']
init_commands_lists = my_agg.memory['init_commands_lists']
posterior_dicts = my_agg.memory['posterior_dicts']
expected_predictive_prob = my_agg.memory['expected_predictive_prob']
A_list = my_agg.memory['A_list']
E_list = my_agg.memory['E_list']
seq_rank_list = my_agg.memory['seq_rank_list']
cluster_num = my_agg.memory['cluster_num']

In [28]:
score_list = my_agg.predict(msdat[int(n_session*0.9):])

In [29]:
print('Average score for ', end='')
for depth in range(1,5):
    print('depth {}: {:.5f},'.format(depth-1, find_score(score_list, depth)), end=' ')

Average score for depth 0: 0.28990, depth 1: 0.44326, depth 2: 0.46949, depth 3: 0.49079, 

### Fifth

$\alpha_{cluster} = \alpha_{label} = 0.1$

$Decision Threshold = 0.5$

$Maximum Depth = 8$

In [33]:
my_agg = my_aggclustering(msdat[:int(n_session*0.9)], 0.1, 0.5, 0.1, 8)
my_agg.fit()
with open('data/aggclustering2_9010_8depth.pkl', 'wb') as f:
    pickle.dump(my_agg, f)

Depth 0 started.
Depth 0 finished. Time spent: 65.49s.
Depth 1 started.
Depth 1 finished. Time spent: 560.44s.
Depth 2 started.
Depth 2 finished. Time spent: 884.66s.
Depth 3 started.
Depth 3 finished. Time spent: 1090.74s.
Depth 4 started.
Depth 4 finished. Time spent: 1105.26s.
Depth 5 started.
Depth 5 finished. Time spent: 1814.51s.
Depth 6 started.
Depth 6 finished. Time spent: 2149.67s.
Depth 7 started.
Depth 7 finished. Time spent: 2153.78s.


In [34]:
with open('data/aggclustering2_9010_8depth.pkl', 'rb') as f:
    my_agg = pickle.load(f)
clustering = my_agg.memory['clustering']
init_commands_lists = my_agg.memory['init_commands_lists']
posterior_dicts = my_agg.memory['posterior_dicts']
expected_predictive_prob = my_agg.memory['expected_predictive_prob']
A_list = my_agg.memory['A_list']
E_list = my_agg.memory['E_list']
seq_rank_list = my_agg.memory['seq_rank_list']
cluster_num = my_agg.memory['cluster_num']

In [36]:
score_list = my_agg.predict(msdat[int(n_session*0.9):])

In [38]:
print('Average score for ', end='')
for depth in range(1,9):
    print('depth {}: {:.5f},'.format(depth-1, find_score(score_list, depth)), end=' ')

Average score for depth 0: 0.28989, depth 1: 0.44354, depth 2: 0.46997, depth 3: 0.49107, depth 4: 0.49377, depth 5: 0.49923, depth 6: 0.49957, depth 7: 0.49984, 